# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import colorsys

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [11]:
import utilities.helper
import trodes.read_exported

In [12]:
FONTSIZE = 20

In [13]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

In [14]:
EPHYS_SAMPLE_RATE = 20000

Explanation of each input and where it comes from.

In [15]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_3_alone_comp"

In [16]:
# TRIAL_LABELS_DF = pd.read_excel("/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_comp_ext/results/2024_06_26_sleap_clustering/data/rce_pilot_3_alone_comp_per_video_trial_labels.xlsx")
TRIAL_LABELS_DF = pd.read_excel("../../data/{}_per_video_trial_labels.xlsx".format(OUTPUT_PREFIX))

In [17]:
LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_04_spectral_and_sleap.pkl".format(OUTPUT_PREFIX))

In [18]:
FULL_LFP_TRACES_PKL = "{}_10_per_trial_spectral_bans_sleap.pkl".format(OUTPUT_PREFIX)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [19]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [20]:
LFP_SPECTRAL_DF.columns

Index(['video_name', 'current_subject', '10s_after_tone_frame',
       '10s_before_tone_frame', 'agent_locations', 'agent_nose',
       'agent_nose_to_reward_port', 'agent_tail_base', 'agent_thorax',
       'agent_thorax_velocity',
       ...
       'vHPC_gamma_phase', 'vHPC_lfp_trace',
       'vHPC_mPFC_granger_all_frequencies_all_windows',
       'vHPC_power_all_frequencies_all_windows', 'vHPC_power_gamma',
       'vHPC_power_theta', 'vHPC_theta_band', 'vHPC_theta_phase',
       'video_timestamps', 'cluster_timestamp'],
      dtype='object', length=220)

In [21]:
TRIAL_LABELS_DF.columns

Index(['Unnamed: 0', 'session_dir', 'tone_start_frame', 'reward_start',
       'reward_dispensed', 'tone_stop_frame', 'condition ',
       'competition_closeness', 'get_reward_frame', 'out_reward_frame',
       'notes', 'experiment', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'all_subjects',
       'first_timestamp', 'last_timestamp', 'tone_start_timestamp',
       'tone_stop_timestamp', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps'],
      dtype='object')

## Merging the labels and LFP spectral

In [22]:
TRIAL_LABELS_DF.head()

,Unnamed: 0,session_dir,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,...,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,0.0,80,100.0,74.0,NaN,NaN,NaN,NaN,...,[ 0 74],[ 0 74],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,0.0,75474.0,[ 0 75474],[ 0 75474]
1,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,1272.0,1352,1372.0,1471.0,rewarded,NaN,NaN,NaN,...,[1282 1283] [1285 1297] [1326 1344] [1362 1509],[1326 1328] [1331 1347] [1380 1592],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,1275889.0,1475891.0,[1285888 1286688] [1287488 1300689] [1330286 1...,[1329489 1331487] [1334489 1350089] [1383490 1...
2,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,3268.0,3348,3368.0,3469.0,rewarded,NaN,NaN,NaN,...,[3281 3324] [3324 3327] [3344 3347] [3347 3523],[3327 3704],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,3275911.0,3475914.0,[3287914 3330314] [3331114 3333712] [3351715 3...,[3333515 3712519]
3,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,4466.0,4546,4566.0,4667.0,rewarded,NaN,NaN,NaN,...,[4441 4476] [4476 4489] [4508 4509] [4511 4534...,[4445 4473] [4494 4551] [4566 4830],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,4475927.0,4675931.0,[4450528 4485129] [4485929 4497529] [4516929 4...,[4454926 4483129] [4503531 4560730] [4575330 4...
4,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,5465.0,5545,5565.0,5664.0,rewarded,NaN,NaN,NaN,...,[5483 5512] [5552 5698],[5455 5486] [5581 5794],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,5475942.0,5675942.0,[5493742 5523542] [5563742 5709147],[5466941 5496741] [5592343 5806546]


In [23]:
LFP_SPECTRAL_DF.head()

,video_name,current_subject,10s_after_tone_frame,10s_before_tone_frame,agent_locations,agent_nose,agent_nose_to_reward_port,agent_tail_base,agent_thorax,agent_thorax_velocity,...,vHPC_gamma_phase,vHPC_lfp_trace,vHPC_mPFC_granger_all_frequencies_all_windows,vHPC_power_all_frequencies_all_windows,vHPC_power_gamma,vHPC_power_theta,vHPC_theta_band,vHPC_theta_phase,video_timestamps,cluster_timestamp
0,20240320_142408_alone_comp_subj_3-1_and_3-3.1,3.1,"[34640, 34640, 34640, 34640, 34640, 34640, 346...","[34040, 34040, 34040, 34040, 34040, 34040, 340...","[[[54.047201234350496, 32.48462136905939], [52...","[[57.31849083590113, 33.645587310504695], [56....","[4.960474, 5.34718, 5.802983, 6.407976, 7.1417...","[[46.630318040576064, 30.523901100123272], [46...","[[48.61806735273943, 32.66608744832453], [48.1...","[7.0735817, 9.794592, 12.977724, 16.4886, 20.1...",...,"[2.1274095, 2.4930122, 2.7189336, 3.0126748, -...","[65.715, 96.329994, 95.159996, 77.61, 112.905,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[0.8526041, 0.89195466, 0.93002737, 0.96665066...","[-1.0124702, -0.7600434, -0.73781365, -0.59900...","[1384, 2770, 4156, 4156, 5542, 6928, 6928, 831...","[34077145, 34077145, 34078531, 34079917, 34079..."
1,20240320_142408_alone_comp_subj_3-1_and_3-3.1,3.3,"[34640, 34640, 34640, 34640, 34640, 34640, 346...","[34040, 34040, 34040, 34040, 34040, 34040, 340...","[[[62.81573570925102, 33.49031610207749], [59....","[[60.5507800219845, 35.26465151447588], [60.78...","[0.53517926, 0.50113046, 0.48567954, 0.4737239...","[[66.06126838242345, 28.880162680346388], [66....","[[64.05699608055423, 31.17933144091776], [64.0...","[0.07300971, 0.08296164, 0.08879429, 0.0909930...",...,"[1.1501461, 1.6517024, 1.9768442, 2.2265384, 2...","[-100.619995, -102.96, -129.87, -165.75, -191....","[[2.5391848, 1.5146569, 1.0411696, 1.5689967, ...","[[0.001042456, 0.009847558, 0.012313993, 0.009...","[0.0008264334, 0.0009773864, 0.0009832539, nan...","[0.0408452, 0.04246293, 0.04184469, nan, nan, ...","[-0.347932, -0.33184236, -0.31452566, -0.29600...","[2.6819186, -2.904363, -2.827914, -2.5523405, ...","[1384, 2770, 4156, 4156, 5542, 6928, 6928, 831...","[34077145, 34077145, 34078531, 34079917, 34079..."
2,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,"[34642, 34642, 34642, 34642, 34642, 34642, 346...","[34042, 34042, 34042, 34042, 34042, 34042, 340...","[[[56.40002042465876, 33.47263135770957], [55....","[[60.438020774390075, 35.57292836881663], [60....","[1.4890755, 1.1949695, 0.93336076, 0.738537, 0...","[[48.80137642581422, 31.16922356981376], [49.0...","[[51.35546426976598, 33.93773101458849], [52.3...","[10.536535, 10.738277, 10.965685, 11.176429, 1...",...,"[0.023950227, 0.96471804, 1.2889154, 1.6205887...","[-74.685, -113.49, -156.78, -194.21999, -266.1...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[0.0013803464, 0.0079310145, 0.009366615, 0.0...","[0.00366902, nan, nan, 0.002184509, 0.00216185...","[0.046842232, nan, nan, 0.023685867, 0.0444350...","[-0.37066078, -0.35752764, -0.3432352, -0.3277...","[2.7155704, -2.8761566, -2.806554, -2.548516, ...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084..."
3,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.3,"[34642, 34642, 34642, 34642, 34642, 34642, 346...","[34042, 34042, 34042, 34042, 34042, 34042, 340...","[[[61.39644296346538, 31.82081289424359], [58....","[[62.13302320168461, 34.13495024512682], [61.3...","[2.1410906, 2.5980964, 3.2073143, 3.9916637, 4...","[[50.83958858240558, 27.427984293098355], [50....","[[54.225662928143386, 30.269937128249303], [54...","[5.105928, 5.1050973, 5.082959, 5.072279, 5.09...",...,"[2.0565047, 2.3807256, 2.6063833, 2.87997, 3.1...","[53.234997, 80.145, 106.079994, 139.42499, 206...","[[2.7552054, 1.5484817, 2.318468, 0.67259854, ...

# Dropping redundant columns

In [24]:
dropping_columns = list(set(TRIAL_LABELS_DF.columns).intersection(set(LFP_SPECTRAL_DF.columns)))

In [25]:
dropping_columns

['video_name',
 'last_timestamp',
 'all_subjects',
 'first_timestamp',
 'tone_start_frame',
 'box_2_port_entry_timestamps',
 'tone_stop_frame',
 'out_reward_frame',
 'get_reward_frame',
 'box_1_port_entry_frames',
 'experiment',
 'condition ',
 'box_1_port_entry_timestamps',
 'notes',
 'competition_closeness',
 'tone_stop_timestamp',
 'session_dir',
 'box_2_port_entry_frames',
 'tone_start_timestamp']

In [26]:
dropping_columns = ['tone_start_frame', 'tone_stop_frame', 'condition ', 'notes', 'competition_closeness', '10s_before_tone_frame', '10s_after_tone_frame']

In [27]:
LFP_SPECTRAL_DF = LFP_SPECTRAL_DF.drop(columns=dropping_columns, errors="ignore")

In [28]:
dropping_columns = list(set(TRIAL_LABELS_DF.columns).intersection(set(LFP_SPECTRAL_DF.columns)))

In [29]:
LFP_SPECTRAL_DF["first_timestamp"].iloc[0]

1830734.0

In [30]:
dropping_columns

['video_name',
 'last_timestamp',
 'all_subjects',
 'tone_stop_timestamp',
 'first_timestamp',
 'out_reward_frame',
 'get_reward_frame',
 'session_dir',
 'box_2_port_entry_frames',
 'experiment',
 'tone_start_timestamp',
 'box_2_port_entry_timestamps',
 'box_1_port_entry_frames',
 'box_1_port_entry_timestamps']

In [31]:
dropping_columns = ['experiment',
                    'box_1_port_entry_frames'
                    'box_1_port_entry_timestamps'
                    'box_2_port_entry_frames'
                    'box_2_port_entry_timestamps',
                    'session_dir',
                    'all_subjects',
                    'first_timestamp',
                    'last_timestamp']


In [32]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.drop(columns=dropping_columns, errors="ignore")

## Merging the LFP and trial dataframe

In [33]:
LFP_SPECTRAL_DF["video_name"] = LFP_SPECTRAL_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))

In [34]:
LFP_SPECTRAL_DF["current_subject"] = LFP_SPECTRAL_DF["current_subject"].astype(str)


In [35]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["video_name"])

In [36]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["condition "])

In [37]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF[~TRIAL_LABELS_DF["condition "].isin(["rewarded"])].copy()

In [38]:
TRIAL_LABELS_DF["video_name"] = TRIAL_LABELS_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))


In [39]:
TRIAL_LABELS_DF["video_name"].unique()

array(['20240320_142408_alone_comp_subj_3-1_and_3-3.1',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.1',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.1',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.1',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.1',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.1',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.1'], dtype=object)

In [40]:
LFP_SPECTRAL_DF["video_name"].unique()

array(['20240320_142408_alone_comp_subj_3-1_and_3-3.1',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.1',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.1',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.1',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.1',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.1',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.1'], dtype=object)

In [41]:
TRIALS_AND_SPECTRAL_DF = pd.merge(TRIAL_LABELS_DF, LFP_SPECTRAL_DF, on=["video_name"], how="inner", suffixes=('', '_remove'))

In [42]:
TRIALS_AND_SPECTRAL_DF[[col for col in TRIALS_AND_SPECTRAL_DF if "_remove" in col]].head()

,get_reward_frame_remove,out_reward_frame_remove,tone_start_timestamp_remove,tone_stop_timestamp_remove,box_1_port_entry_frames_remove,box_1_port_entry_timestamps_remove,box_2_port_entry_frames_remove,box_2_port_entry_timestamps_remove
0,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34276299.0, 34276299.0, 34276299.0, 34276299....","[34476299.0, 34476299.0, 34476299.0, 34476299....","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 75474], [153075, 158874], [252873, 306276...","[[0, 74], [160, 225], [254, 257], [257, 279], ...","[[0, 75474], [162074, 226073], [255076, 258076..."
1,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34276299.0, 34276299.0, 34276299.0, 34276299....","[34476299.0, 34476299.0, 34476299.0, 34476299....","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 75474], [153075, 158874], [252873, 306276...","[[0, 74], [160, 225], [254, 257], [257, 279], ...","[[0, 75474], [162074, 226073], [255076, 258076..."
2,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34276299.0, 34276299.0, 34276299.0, 34276299....","[34476299.0, 34476299.0, 34476299.0, 34476299....","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 75474], [153075, 158874], [252873, 306276...","[[0, 74], [160, 225], [254, 257], [257, 279], ...","[[0, 75474], [162074, 226073], [255076, 258076..."
3,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34276299.0, 34276299.0, 34276299.0, 34276299....","[34476299.0, 34476299.0, 34476299.0, 34476299....","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 75474], [153075, 158874], [252873, 306276...","[[0, 74], [160, 225], [254, 257], [257, 279], ...","[[0, 75474], [162074, 226073], [255076, 258076..."
4,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34276299.0, 34276299.0, 34276299.0, 34276299....","[34476299.0, 34476299.0, 34476299.0, 34476299....","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 75474], [153075, 158874], [252873, 306276...","[[0, 74], [160, 225], [254, 257], [257, 279], ...","[[0, 75474], [162074, 226073], [255076, 258076..."


In [43]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF if "remove" in col], errors="ignore")

In [44]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF if "Unnamed" in col])

In [45]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,notes,box_1_port_entry_frames,...,vHPC_gamma_phase,vHPC_lfp_trace,vHPC_mPFC_granger_all_frequencies_all_windows,vHPC_power_all_frequencies_all_windows,vHPC_power_gamma,vHPC_power_theta,vHPC_theta_band,vHPC_theta_phase,video_timestamps,cluster_timestamp
0,34240.0,34320,34340.0,34440.0,3.1,Subj 1 Only,NaN,NaN,NaN,[34236 34249] [34270 34280] [34342 34483],...,"[2.1274095, 2.4930122, 2.7189336, 3.0126748, -...","[65.715, 96.329994, 95.159996, 77.61, 112.905,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[0.8526041, 0.89195466, 0.93002737, 0.96665066...","[-1.0124702, -0.7600434, -0.73781365, -0.59900...","[1384, 2770, 4156, 4156, 5542, 6928, 6928, 831...","[34077145, 34077145, 34078531, 34079917, 34079..."
1,34240.0,34320,34340.0,34440.0,3.1,Subj 1 Only,NaN,NaN,NaN,[34236 34249] [34270 34280] [34342 34483],...,"[1.1501461, 1.6517024, 1.9768442, 2.2265384, 2...","[-100.619995, -102.96, -129.87, -165.75, -191....","[[2.5391848, 1.5146569, 1.0411696, 1.5689967, ...","[[0.001042456, 0.009847558, 0.012313993, 0.009...","[0.0008264334, 0.0009773864, 0.0009832539, nan...","[0.0408452, 0.04246293, 0.04184469, nan, nan, ...","[-0.347932, -0.33184236, -0.31452566, -0.29600...","[2.6819186, -2.904363, -2.827914, -2.5523405, ...","[1384, 2770, 4156, 4156, 5542, 6928, 6928, 831...","[34077145, 34077145, 34078531, 34079917, 34079..."
2,35238.0,35318,35338.0,35437.0,3.1,Subj 1 then Subj 2,NaN,NaN,NaN,[35237 35255] [35280 35312] [35324 35368] [353...,...,"[2.1274095, 2.4930122, 2.7189336, 3.0126748, -...","[65.715, 96.329994, 95.159996, 77.61, 112.905,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[0.8526041, 0.89195466, 0.93002737, 0.96665066...","[-1.0124702, -0.7600434, -0.73781365, -0.59900...","[1384, 2770, 4156, 4156, 5542, 6928, 6928, 831...","[34077145, 34077145, 34078531, 34079917, 34079..."
3,35238.0,35318,35338.0,35437.0,3.1,Subj 1 then Subj 2,NaN,NaN,NaN,[35237 35255] [35280 35312] [35324 35368] [353...,...,"[1.1501461, 1.6517024, 1.9768442, 2.2265384, 2...","[-100.619995, -102.96, -129.87, -165.75, -191....","[[2.5391848, 1.5146569, 1.0411696, 1.5689967, ...","[[0.001042456, 0.009847558, 0.012313993, 0.009...","[0.0008264334, 0.0009773864, 0.0009832539, nan...","[0.0408452, 0.04246293, 0.04184469, nan, nan, ...","[-0.347932, -0.33184236, -0.31452566, -0.29600...","[2.6819186, -2.904363, -2.827914, -2.5523405, ...","[1384, 2770, 4156, 4156, 5542, 6928, 6928, 831...","[34077145, 34077145, 34078531, 34079917, 34079..."
4,36337.0,36417,36437.0,36536.0,3.3,Subj 2 Only,NaN,NaN,NaN,[36312 36345] [36439 36503] [36522 36562],...,"[2.1274095, 2.4930122, 2.7189336, 3.0126748, -...","[65.715, 96.329994, 95.159996, 77.61, 112.905,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[0.8526041, 0.89195466, 0.93002737, 0.96665066...","[-1.0124702, -0.7600434, -0.73781365, -0.59900...","[1384, 2770, 4156, 4156, 5542, 6928, 6928, 831...","[34077145, 34077145, 34078531, 34079917, 34079..."


In [46]:
TRIALS_AND_SPECTRAL_DF.tail()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,notes,box_1_port_entry_frames,...,vHPC_gamma_phase,vHPC_lfp_trace,vHPC_mPFC_granger_all_frequencies_all_windows,vHPC_power_all_frequencies_all_windows,vHPC_power_gamma,vHPC_power_theta,vHPC_theta_band,vHPC_theta_phase,video_timestamps,cluster_timestamp
283,57880.0,57960,57980.0,58079.0,4.4,Subj 2 then Subj 1,NaN,NaN,NaN,[57883 57894] [57896 57900] [57901 57909] [579...,...,"[0.45023292, 1.3117398, 1.6877564, 1.9906867, ...","[45.045, 26.91, -14.82, -51.675, -58.695, -51....","[[0.6611806, 0.2473011, 0.73096365, 0.5656533,...","[[0.00643258, 0.006992285, 0.0069811256, 0.004...","[0.00016188143, 0.0002133701, 0.00021314951, 0...","[0.007144761, 0.0037135014, 0.005482981, 0.006...","[-0.096212715, -0.10505088, -0.11372743, -0.12...","[2.032272, 2.214738, 2.247664, 2.3557777, 2.39...","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[37164862, 37166248, 37167634, 37167634, 37169..."
284,60077.0,60157,60177.0,60276.0,4.4,Subj 2 blocking Subj 1,NaN,NaN,4.2 just jumping around on the right side of port,[60070 60098] [60099 60101] [60103 60105] [601...,...,"[0.72235066, 1.454637, 1.8199799, 2.1028404, 2...","[-9.75, 7.41, 14.624999, -12.285, -66.299995, ...","[[1.095963, 0.27759162, 0.13746454, 0.27555922...","[[0.007860632, 0.0064592217, 0.006611264, 0.00...","[0.003164349, 0.0026817215, 0.002723993, 0.002...","[0.014044589, 0.025243934, 0.021230606, 0.0120...","[-0.16617914, -0.17804804, -0.18976012, -0.201...","[2.005712, 2.1644444, 2.1874144, 2.27717, 2.30...","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[37164862, 37166248, 37167634, 37167634, 37169..."
285,60077.0,60157,60177.0,60276.0,4.4,Subj 2 blocking Subj 1,NaN,NaN,4.2 just jumping around on the right side of port,[60070 60098] [60099 60101] [60103 60105] [601...,...,"[0.45023292, 1.3117398, 1.6877564, 1.9906867, ...","[45.045, 26.91, -14.82, -51.675, -58.695, -51....","[[0.6611806, 0.2473011, 0.73096365, 0.5656533,...","[[0.00643258, 0.006992285, 0.0069811256, 0.004...","[0.00016188143, 0.0002133701, 0.00021314951, 0...","[0.007144761, 0.0037135014, 0.005482981, 0.006...","[-0.096212715, -0.10505088, -0.11372743, -0.12...","[2.032272, 2.214738, 2.247664, 2.3557777, 2.39...","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[37164862, 37166248, 37167634, 37167634, 37169..."
286,61174.0,61254,61274.0,61375.0,tie,Close Call,NaN,NaN,NaN,[61188 61211] [61211 61214] [61214 61220] [612...,...,"[0.72235066, 1.454637, 1.8199799, 2.1028404, 2...","[-9.75, 7.41, 14.624999, -12.285, -66.299995, ...","[[1.095963, 0.27759162, 0.13746454, 0.27555922...","[[0.007860632, 0.0064592217, 0.006611264, 0.00...","[0.003164349, 0.0026817215, 0.002723993, 0.002...","[0.014044589, 0.025243934, 0.021230606, 0.0120...","[-0.16617914, -0.17804804, -0.18976012, -0.201...","[2.005712, 2.1644444, 2.1874144, 2.27717, 2.30...","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[37164862, 37166248, 37167634, 37167634, 37169..."
287,61174.0,61254,61274.0,61375.0,tie,Close Call,NaN,NaN,NaN,[61188 61211] [61211 61214] [61214 61220] [612...,...,"[0.45023292, 1.3117398, 1.6877564, 1.9906867, ...","[45.045, 26.91, -14.82, -51.675, -58.695, -51....","[[0.6611806, 0.2473011, 0.73096365, 0.5656533,...","[[0.00643258, 0.006992285, 0.0069811256, 0.004...","[0.00016188143, 0.0002133701, 0.00021314951, 0...","[0.007144761, 0.0037135014, 0.005482981, 0.006...","[-0.096212715, -0.10505088, -0.11372743, -0.12...","[2.032272, 2.214738, 2.247664, 2.3557777, 2.39...","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[37164862, 37166248, 37167634, 37167634, 37169..."


In [47]:
timestamps_col = [col for col in TRIALS_AND_SPECTRAL_DF if "timestamp" in col and "timestamps" not in col and "cluster" not in col]

In [48]:
for col in timestamps_col:
    TRIALS_AND_SPECTRAL_DF[col] = TRIALS_AND_SPECTRAL_DF[col].astype(int)

In [49]:
TRIALS_AND_SPECTRAL_DF[timestamps_col].head()

,tone_start_timestamp,tone_stop_timestamp,first_timestamp,last_timestamp
0,34276299,34476299,1830734,65425515
1,34276299,34476299,1830734,65425515
2,35276311,35476311,1830734,65425515
3,35276311,35476311,1830734,65425515
4,36376322,36576327,1830734,65425515


# Calculating the relevant timestamps

In [50]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] - 10 * EPHYS_SAMPLE_RATE

In [51]:
TRIALS_AND_SPECTRAL_DF["post_trial_end_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"] + 10 * EPHYS_SAMPLE_RATE

In [52]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    34276299
1    34276299
2    35276311
3    35276311
4    36376322
Name: tone_start_timestamp, dtype: int64

In [53]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    34276299
1    34276299
2    35276311
3    35276311
4    36376322
Name: tone_start_timestamp, dtype: int64

In [54]:
TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"].head()

0    34476299
1    34476299
2    35476311
3    35476311
4    36576327
Name: tone_stop_timestamp, dtype: int64

In [55]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,notes,box_1_port_entry_frames,...,vHPC_mPFC_granger_all_frequencies_all_windows,vHPC_power_all_frequencies_all_windows,vHPC_power_gamma,vHPC_power_theta,vHPC_theta_band,vHPC_theta_phase,video_timestamps,cluster_timestamp,baseline_start_timestamp,post_trial_end_timestamp
0,34240.0,34320,34340.0,34440.0,3.1,Subj 1 Only,NaN,NaN,NaN,[34236 34249] [34270 34280] [34342 34483],...,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[0.8526041, 0.89195466, 0.93002737, 0.96665066...","[-1.0124702, -0.7600434, -0.73781365, -0.59900...","[1384, 2770, 4156, 4156, 5542, 6928, 6928, 831...","[34077145, 34077145, 34078531, 34079917, 34079...",34076299,34676299
1,34240.0,34320,34340.0,34440.0,3.1,Subj 1 Only,NaN,NaN,NaN,[34236 34249] [34270 34280] [34342 34483],...,"[[2.5391848, 1.5146569, 1.0411696, 1.5689967, ...","[[0.001042456, 0.009847558, 0.012313993, 0.009...","[0.0008264334, 0.0009773864, 0.0009832539, nan...","[0.0408452, 0.04246293, 0.04184469, nan, nan, ...","[-0.347932, -0.33184236, -0.31452566, -0.29600...","[2.6819186, -2.904363, -2.827914, -2.5523405, ...","[1384, 2770, 4156, 4156, 5542, 6928, 6928, 831...","[34077145, 34077145, 34078531, 34079917, 34079...",34076299,34676299
2,35238.0,35318,35338.0,35437.0,3.1,Subj 1 then Subj 2,NaN,NaN,NaN,[35237 35255] [35280 35312] [35324 35368] [353...,...,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[0.8526041, 0.89195466, 0.93002737, 0.96665066...","[-1.0124702, -0.7600434, -0.73781365, -0.59900...","[1384, 2770, 4156, 4156, 5542, 6928, 6928, 831...","[34077145, 34077145, 34078531, 34079917, 34079...",35076311,35676311
3,35238.0,35318,35338.0,35437.0,3.1,Subj 1 then Subj 2,NaN,NaN,NaN,[35237 35255] [35280 35312] [35324 35368] [353...,...,"[[2.5391848, 1.5146569, 1.0411696, 1.5689967, ...","[[0.001042456, 0.009847558, 0.012313993, 0.009...","[0.0008264334, 0.0009773864, 0.0009832539, nan...","[0.0408452, 0.04246293, 0.04184469, nan, nan, ...","[-0.347932, -0.33184236, -0.31452566, -0.29600...","[2.6819186, -2.904363, -2.827914, -2.5523405, ...","[1384, 2770, 4156, 4156, 5542, 6928, 6928, 831...","[34077145, 34077145, 34078531, 34079917, 34079...",35076311,35676311
4,36337.0,36417,36437.0,36536.0,3.3,Subj 2 Only,NaN,NaN,NaN,[36312 36345] [36439 36503] [36522 36562],...,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[0.8526041, 0.89195466, 0.93002737, 0.96665066...","[-1.0124702, -0.7600434, -0.73781365, -0.59900...","[1384, 2770, 4156, 4156, 5542, 6928, 6928, 831...","[34077145, 34077145, 34078531, 34079917, 34079...",36176322,36776327


# Filtering out the SLEAP posed for during trials

In [56]:
TRIALS_AND_SPECTRAL_DF["video_frame"] = TRIALS_AND_SPECTRAL_DF["video_timestamps"].apply(lambda x: np.array(list(range(len(x)))) + 1)

In [57]:
TRIALS_AND_SPECTRAL_DF["video_frame"].head().apply(lambda x: x.shape)

0    (63504,)
1    (63504,)
2    (63504,)
3    (63504,)
4    (63504,)
Name: video_frame, dtype: object

In [58]:
['subject_thorax_to_agent_thorax',
 'nose_to_reward_port_sum',
 'nose_to_reward_port_diff',
 'thorax_velocity_sum',
 'thorax_velocity_diff',
 'to_reward_port_angle_sum',
 'to_reward_port_angle_diff',
 'subject_nose_to_reward_port',
 'subject_thorax_velocity',
 'subject_to_reward_port_angle',
 'agent_nose_to_reward_port',
 'agent_thorax_velocity',
 'agent_to_reward_port_angle',
 'closebool_subject_nose_to_reward_port',
 'closebool_agent_nose_to_reward_port',
 'movingbool_subject_thorax_velocity',
 'movingbool_agent_thorax_velocity',
 'manual_cluster_id',
 'standard_embedding_x',
 'standard_embedding_y',
 'kmeans_cluster',
 'subject_locations',
 'agent_locations',
 'subject_thorax',
 'subject_nose',
 'subject_tail_base',
 'agent_thorax',
 'agent_nose',
 'agent_tail_base']

['subject_thorax_to_agent_thorax',
 'nose_to_reward_port_sum',
 'nose_to_reward_port_diff',
 'thorax_velocity_sum',
 'thorax_velocity_diff',
 'to_reward_port_angle_sum',
 'to_reward_port_angle_diff',
 'subject_nose_to_reward_port',
 'subject_thorax_velocity',
 'subject_to_reward_port_angle',
 'agent_nose_to_reward_port',
 'agent_thorax_velocity',
 'agent_to_reward_port_angle',
 'closebool_subject_nose_to_reward_port',
 'closebool_agent_nose_to_reward_port',
 'movingbool_subject_thorax_velocity',
 'movingbool_agent_thorax_velocity',
 'manual_cluster_id',
 'standard_embedding_x',
 'standard_embedding_y',
 'kmeans_cluster',
 'subject_locations',
 'agent_locations',
 'subject_thorax',
 'subject_nose',
 'subject_tail_base',
 'agent_thorax',
 'agent_nose',
 'agent_tail_base']

In [59]:
# sleap_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "locations" in col or "velocity" in col or "to_reward_port" in col or "video_frame" in col]

In [60]:
sleap_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if ("thorax" in col or "nose" in col or "standard_embedding" in col or "cluster" in col or "frame_index" in col or "locations" in col or "tailbase" in col or "competitiveness" in col) and "timestamp" not in col]


In [61]:
sleap_columns

['agent_locations',
 'agent_nose',
 'agent_nose_to_reward_port',
 'agent_thorax',
 'agent_thorax_velocity',
 'closebool_agent_nose_to_reward_port',
 'closebool_subject_nose_to_reward_port',
 'clusterable_embedding_x',
 'clusterable_embedding_y',
 'competitiveness',
 'frame_index',
 'kmeans_cluster',
 'manual_cluster_id',
 'movingbool_agent_thorax_velocity',
 'movingbool_subject_thorax_velocity',
 'nose_to_reward_port_diff',
 'nose_to_reward_port_sum',
 'nose_to_tail_diff',
 'nose_to_tail_sum',
 'standard_embedding_x',
 'standard_embedding_y',
 'subject_locations',
 'subject_nose',
 'subject_nose_to_reward_port',
 'subject_thorax',
 'subject_thorax_to_agent_thorax',
 'subject_thorax_velocity',
 'thorax_velocity_diff',
 'thorax_velocity_sum',
 'within_trial_frame_index']

In [62]:
for col in sleap_columns:
    print(col)
    print(TRIALS_AND_SPECTRAL_DF[col].apply(lambda x: x.shape).iloc[0])

agent_locations
(12589, 6, 2)
agent_nose
(12589, 2)
agent_nose_to_reward_port
(12589,)
agent_thorax
(12589, 2)
agent_thorax_velocity
(12589,)
closebool_agent_nose_to_reward_port
(12589,)
closebool_subject_nose_to_reward_port
(12589,)
clusterable_embedding_x
(12589,)
clusterable_embedding_y
(12589,)
competitiveness
(12589,)
frame_index
(12589,)
kmeans_cluster
(12589,)
manual_cluster_id
(12589,)
movingbool_agent_thorax_velocity
(12589,)
movingbool_subject_thorax_velocity
(12589,)
nose_to_reward_port_diff
(12589,)
nose_to_reward_port_sum
(12589,)
nose_to_tail_diff
(12589,)
nose_to_tail_sum
(12589,)
standard_embedding_x
(12589,)
standard_embedding_y
(12589,)
subject_locations
(12589, 6, 2)
subject_nose
(12589, 2)
subject_nose_to_reward_port
(12589,)
subject_thorax
(12589, 2)
subject_thorax_to_agent_thorax
(12589,)
subject_thorax_velocity
(12589,)
thorax_velocity_diff
(12589,)
thorax_velocity_sum
(12589,)
within_trial_frame_index
(12589,)


In [63]:
TRIALS_AND_SPECTRAL_DF["cluster_timestamp"].iloc[0]

array([34077145, 34077145, 34078531, ..., 61673966, 61675351, 61676737],
      dtype=int32)

In [64]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"]

0      34276299
1      34276299
2      35276311
3      35276311
4      36376322
         ...   
283    57964893
284    60164920
285    60164920
286    61264933
287    61264933
Name: tone_start_timestamp, Length: 288, dtype: int64

In [65]:
for col in sorted(sleap_columns):
    updated_item_col = "trial_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_and_post_video_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["cluster_timestamp"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["cluster_timestamp"], items=x[col])[0], axis=1)

trial_and_post_agent_locations
trial_and_post_agent_nose
trial_and_post_agent_nose_to_reward_port
trial_and_post_agent_thorax
trial_and_post_agent_thorax_velocity
trial_and_post_closebool_agent_nose_to_reward_port
trial_and_post_closebool_subject_nose_to_reward_port
trial_and_post_clusterable_embedding_x
trial_and_post_clusterable_embedding_y
trial_and_post_competitiveness
trial_and_post_frame_index
trial_and_post_kmeans_cluster
trial_and_post_manual_cluster_id
trial_and_post_movingbool_agent_thorax_velocity
trial_and_post_movingbool_subject_thorax_velocity
trial_and_post_nose_to_reward_port_diff
trial_and_post_nose_to_reward_port_sum
trial_and_post_nose_to_tail_diff
trial_and_post_nose_to_tail_sum
trial_and_post_standard_embedding_x
trial_and_post_standard_embedding_y
trial_and_post_subject_locations
trial_and_post_subject_nose
trial_and_post_subject_nose_to_reward_port
trial_and_post_subject_thorax
trial_and_post_subject_thorax_to_agent_thorax
trial_and_post_subject_thorax_velocity
t

In [66]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,notes,box_1_port_entry_frames,...,trial_and_post_subject_locations,trial_and_post_subject_nose,trial_and_post_subject_nose_to_reward_port,trial_and_post_subject_thorax,trial_and_post_subject_thorax_to_agent_thorax,trial_and_post_subject_thorax_velocity,trial_and_post_thorax_velocity_diff,trial_and_post_thorax_velocity_sum,trial_and_post_within_trial_frame_index,trial_and_post_video_timestamps
0,34240.0,34320,34340.0,34440.0,3.1,Subj 1 Only,NaN,NaN,NaN,[34236 34249] [34270 34280] [34342 34483],...,"[[[62.57065705282688, 33.44730809206244], [58....","[[60.54818493730512, 35.74634421564697], [60.8...","[0.28861773, 0.17335962, 0.17028531, 0.2535302...","[[59.8319529727617, 31.001132715032707], [59.8...","[12.89103, 12.176237, 11.61231, 11.06019, 10.5...","[-0.021209748, 0.003994167, 0.15215835, 0.4798...","[26.120943, 23.606047, 20.731367, 17.724455, 1...","[27.127563, 25.086893, 22.796099, 20.469105, 1...","[200, 201, 202, 203, 204, 205, 206, 207, 208, ...","[34276710, 34278096, 34279482, 34279482, 34280..."
1,34240.0,34320,34340.0,34440.0,3.1,Subj 1 Only,NaN,NaN,NaN,[34236 34249] [34270 34280] [34342 34483],...,"[[[70.76603142951862, 27.905445222198825], [66...","[[67.33614650729025, 29.551449212479824], [66....","[9.162211, 8.083501, 7.0652747, 6.1474023, 5.3...","[[69.77804561944015, 24.819920337934217], [70....","[12.89103, 12.176237, 11.61231, 11.06019, 10.5...","[27.148773, 25.0829, 22.643942, 19.98921, 17.2...","[26.120943, 23.606047, 20.731367, 17.724455, 1...","[27.127563, 25.086893, 22.796099, 20.469105, 1...","[200, 201, 202, 203, 204, 205, 206, 207, 208, ...","[34276710, 34278096, 34279482, 34279482, 34280..."
2,35238.0,35318,35338.0,35437.0,3.1,Subj 1 then Subj 2,NaN,NaN,NaN,[35237 35255] [35280 35312] [35324 35368] [353...,...,"[[[62.27856458758486, 33.78491219048569], [58....","[[60.821855153064945, 35.12897321756974], [60....","[0.41515765, 0.42165327, 0.43931708, 0.4439942...","[[63.76963180256321, 31.344854927989847], [63....","[12.874714, 12.780252, 12.830084, 12.994184, 1...","[0.5843707, 0.5663497, 0.55942494, 0.56991917,...","[26.742891, 27.9868, 29.002823, 29.783545, 30....","[27.911633, 29.1195, 30.121674, 30.923384, 31....","[200, 201, 202, 203, 204, 205, 206, 207, 208, ...","[35277308, 35277308, 35278694, 35280080, 35280..."
3,35238.0,35318,35338.0,35437.0,3.1,Subj 1 then Subj 2,NaN,NaN,NaN,[35237 35255] [35280 35312] [35324 35368] [353...,...,"[[[59.557187688302, 21.40206507656825], [56.81...","[[56.44453190593808, 22.041576176751878], [54....","[13.304398, 13.279373, 13.2186365, 13.117102, ...","[[62.308924226592204, 17.97009425079981], [62....","[12.874714, 12.780252, 12.830084, 12.994184, 1...","[27.327261, 28.553148, 29.562248, 30.353464, 3...","[26.742891, 27.9868, 29.002823, 29.783545, 30....","[27.911633, 29.1195, 30.121674, 30.923384, 31....","[200, 201, 202, 203, 204, 205, 206, 207, 208, ...","[35277308, 35277308, 35278694, 35280080, 35280..."
4,36337.0,36417,36437.0,36536.0,3.3,Subj 2 Only,NaN,NaN,NaN,[36312 36345] [36439 36503] [36522 36562],...,"[[[62.31907642599725, 33.81738396661642], [60....","[[60.32622249671731, 35.42778006327852], [60.3...","[0.518957, 0.6057527, 0.6832184, 0.77924526, 0...","[[66.78737877919696, 32.823619089907766], [66....","[26.3084, 26.333275, 26.375715, 26.454876, 26....","[1.0364465, 1.3400655, 1.656862, 1.9446498, 2....","[0.672737, 0.96411556, 1.2816203, 1.5727413, 1...","[1.6744555, 1.9707181, 2.2606552, 2.5124037, 2...","[200, 201, 202, 203, 204, 205, 206, 207, 208, ...","[36377688, 36379074, 36380460, 36380460, 36381..."


In [67]:
TRIALS_AND_SPECTRAL_DF["trial_and_post_frame_index"].iloc[0].shape

(399,)

In [68]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=sleap_columns + ["cluster_timestamp"], errors="ignore")

In [69]:
for col in TRIALS_AND_SPECTRAL_DF.columns:
    print(col)

tone_start_frame
reward_start
reward_dispensed
tone_stop_frame
condition 
competition_closeness
get_reward_frame
out_reward_frame
notes
box_1_port_entry_frames
box_2_port_entry_frames
video_name
tone_start_timestamp
tone_stop_timestamp
box_1_port_entry_timestamps
box_2_port_entry_timestamps
current_subject
agent_tail_base
agent_to_reward_port_angle
color
comp_id
subject_tail_base
subject_to_reward_port_angle
to_reward_port_angle_diff
to_reward_port_angle_sum
tone_start_to_stop_frame
within_trial_index
agent
all_subjects
body_parts
box_bottom_left
box_bottom_right
box_number
box_top_left
box_top_right
experiment
first_timestamp
last_timestamp
reward_port
session_dir
sleap_name
start_frame
stop_frame
tracked_subject
video_id
BLA-to-LH_granger_gamma
BLA-to-LH_granger_theta
BLA-to-MD_granger_gamma
BLA-to-MD_granger_theta
BLA-to-mPFC_granger_gamma
BLA-to-mPFC_granger_theta
BLA-to-vHPC_granger_gamma
BLA-to-vHPC_granger_theta
BLA_LH_coherence_all_frequencies_all_windows
BLA_LH_coherence_gamma

- Filtering coherence

In [70]:
coherence_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "coherence" in col and "timestamps" not in col and "calculation" not in col]

In [71]:
coherence_columns

['BLA_LH_coherence_all_frequencies_all_windows',
 'BLA_LH_coherence_gamma',
 'BLA_LH_coherence_theta',
 'BLA_MD_coherence_all_frequencies_all_windows',
 'BLA_MD_coherence_gamma',
 'BLA_MD_coherence_theta',
 'BLA_mPFC_coherence_all_frequencies_all_windows',
 'BLA_mPFC_coherence_gamma',
 'BLA_mPFC_coherence_theta',
 'BLA_vHPC_coherence_all_frequencies_all_windows',
 'BLA_vHPC_coherence_gamma',
 'BLA_vHPC_coherence_theta',
 'LH_MD_coherence_all_frequencies_all_windows',
 'LH_MD_coherence_gamma',
 'LH_MD_coherence_theta',
 'LH_mPFC_coherence_all_frequencies_all_windows',
 'LH_mPFC_coherence_gamma',
 'LH_mPFC_coherence_theta',
 'LH_vHPC_coherence_all_frequencies_all_windows',
 'LH_vHPC_coherence_gamma',
 'LH_vHPC_coherence_theta',
 'MD_mPFC_coherence_all_frequencies_all_windows',
 'MD_mPFC_coherence_gamma',
 'MD_mPFC_coherence_theta',
 'MD_vHPC_coherence_all_frequencies_all_windows',
 'MD_vHPC_coherence_gamma',
 'MD_vHPC_coherence_theta',
 'mPFC_vHPC_coherence_all_frequencies_all_windows',


In [72]:
for col in sorted(coherence_columns):
    updated_item_col = "trial_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_and_post_coherence_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)

trial_and_post_BLA_LH_coherence_all_frequencies_all_windows
trial_and_post_BLA_LH_coherence_gamma
trial_and_post_BLA_LH_coherence_theta
trial_and_post_BLA_MD_coherence_all_frequencies_all_windows
trial_and_post_BLA_MD_coherence_gamma
trial_and_post_BLA_MD_coherence_theta
trial_and_post_BLA_mPFC_coherence_all_frequencies_all_windows
trial_and_post_BLA_mPFC_coherence_gamma
trial_and_post_BLA_mPFC_coherence_theta
trial_and_post_BLA_vHPC_coherence_all_frequencies_all_windows
trial_and_post_BLA_vHPC_coherence_gamma
trial_and_post_BLA_vHPC_coherence_theta
trial_and_post_LH_MD_coherence_all_frequencies_all_windows
trial_and_post_LH_MD_coherence_gamma
trial_and_post_LH_MD_coherence_theta
trial_and_post_LH_mPFC_coherence_all_frequencies_all_windows
trial_and_post_LH_mPFC_coherence_gamma
trial_and_post_LH_mPFC_coherence_theta
trial_and_post_LH_vHPC_coherence_all_frequencies_all_windows
trial_and_post_LH_vHPC_coherence_gamma
trial_and_post_LH_vHPC_coherence_theta
trial_and_post_MD_mPFC_coherence_

In [73]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=coherence_columns + ["coherence_timestamps"], errors="ignore")

- Filtering Grangers

In [74]:
granger_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "granger" in col and "timestamps" not in col and "calculation" not in col]

In [75]:
granger_columns

['BLA-to-LH_granger_gamma',
 'BLA-to-LH_granger_theta',
 'BLA-to-MD_granger_gamma',
 'BLA-to-MD_granger_theta',
 'BLA-to-mPFC_granger_gamma',
 'BLA-to-mPFC_granger_theta',
 'BLA-to-vHPC_granger_gamma',
 'BLA-to-vHPC_granger_theta',
 'BLA_LH_granger_all_frequencies_all_windows',
 'BLA_MD_granger_all_frequencies_all_windows',
 'BLA_mPFC_granger_all_frequencies_all_windows',
 'BLA_vHPC_granger_all_frequencies_all_windows',
 'LH-to-BLA_granger_gamma',
 'LH-to-BLA_granger_theta',
 'LH-to-MD_granger_gamma',
 'LH-to-MD_granger_theta',
 'LH-to-mPFC_granger_gamma',
 'LH-to-mPFC_granger_theta',
 'LH-to-vHPC_granger_gamma',
 'LH-to-vHPC_granger_theta',
 'LH_BLA_granger_all_frequencies_all_windows',
 'LH_MD_granger_all_frequencies_all_windows',
 'LH_mPFC_granger_all_frequencies_all_windows',
 'LH_vHPC_granger_all_frequencies_all_windows',
 'MD-to-BLA_granger_gamma',
 'MD-to-BLA_granger_theta',
 'MD-to-LH_granger_gamma',
 'MD-to-LH_granger_theta',
 'MD-to-mPFC_granger_gamma',
 'MD-to-mPFC_granger_t

In [76]:
for col in sorted(granger_columns):
    updated_item_col = "trial_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_and_post_granger_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)

trial_and_post_BLA-to-LH_granger_gamma
trial_and_post_BLA-to-LH_granger_theta
trial_and_post_BLA-to-MD_granger_gamma
trial_and_post_BLA-to-MD_granger_theta
trial_and_post_BLA-to-mPFC_granger_gamma
trial_and_post_BLA-to-mPFC_granger_theta
trial_and_post_BLA-to-vHPC_granger_gamma
trial_and_post_BLA-to-vHPC_granger_theta
trial_and_post_BLA_LH_granger_all_frequencies_all_windows
trial_and_post_BLA_MD_granger_all_frequencies_all_windows
trial_and_post_BLA_mPFC_granger_all_frequencies_all_windows
trial_and_post_BLA_vHPC_granger_all_frequencies_all_windows
trial_and_post_LH-to-BLA_granger_gamma
trial_and_post_LH-to-BLA_granger_theta
trial_and_post_LH-to-MD_granger_gamma
trial_and_post_LH-to-MD_granger_theta
trial_and_post_LH-to-mPFC_granger_gamma
trial_and_post_LH-to-mPFC_granger_theta
trial_and_post_LH-to-vHPC_granger_gamma
trial_and_post_LH-to-vHPC_granger_theta
trial_and_post_LH_BLA_granger_all_frequencies_all_windows
trial_and_post_LH_MD_granger_all_frequencies_all_windows
trial_and_post_

In [77]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=granger_columns + ["granger_timestamps"], errors="ignore")

In [78]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

- Filtering power

In [79]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,notes,box_1_port_entry_frames,...,trial_and_post_vHPC-to-LH_granger_theta,trial_and_post_vHPC-to-MD_granger_gamma,trial_and_post_vHPC-to-MD_granger_theta,trial_and_post_vHPC-to-mPFC_granger_gamma,trial_and_post_vHPC-to-mPFC_granger_theta,trial_and_post_vHPC_BLA_granger_all_frequencies_all_windows,trial_and_post_vHPC_LH_granger_all_frequencies_all_windows,trial_and_post_vHPC_MD_granger_all_frequencies_all_windows,trial_and_post_vHPC_mPFC_granger_all_frequencies_all_windows,trial_and_post_granger_timestamps
0,34240.0,34320,34340.0,34440.0,3.1,Subj 1 Only,NaN,NaN,NaN,[34236 34249] [34270 34280] [34342 34483],...,"[0.5568959, 0.24519724, 0.20076632, 0.5333081,...","[0.49166852, 0.22800069, 0.28546643, 0.4283831...","[0.45390114, 0.23823264, 0.20773382, 0.9316409...","[0.3317966, 0.34990138, nan, nan, 0.38994804, ...","[0.18835476, 0.2621064, nan, nan, 0.2167768, 0...","[[0.39526093, 0.14897035, 0.25017428, 0.354303...","[[0.45540842, 0.10189814, 0.10806129, 0.155386...","[[0.3080779, 0.10640508, 0.059618432, 0.078086...","[[3.5291126, 0.22613533, 0.3948932, 0.03885741...","[34280000, 34290000, 34300000, 34310000, 34320..."
1,34240.0,34320,34340.0,34440.0,3.1,Subj 1 Only,NaN,NaN,NaN,[34236 34249] [34270 34280] [34342 34483],...,"[nan, nan, nan, nan, nan, 0.18690442, 0.049745...","[nan, nan, nan, nan, nan, 0.24698833, nan, nan...","[nan, nan, nan, nan, nan, 0.19479248, nan, nan...","[0.12839502, 0.19830413, nan, nan, nan, 0.1601...","[0.15178944, 0.5260722, nan, nan, nan, 0.30177...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[0.016699517, 0.22255981, 0.07830153, 0.16324...","[34280000, 34290000, 34300000, 34310000, 34320..."
2,35238.0,35318,35338.0,35437.0,3.1,Subj 1 then Subj 2,NaN,NaN,NaN,[35237 35255] [35280 35312] [35324 35368] [353...,...,"[0.574632, 0.13700777, 0.9124831, 1.6471814, 0...","[0.45921808, 0.4018911, 0.403811, 0.50582206, ...","[0.57140416, 0.22462066, 1.1100769, 1.5301652,...","[0.40667596, 0.15274861, 0.25372973, 0.649355,...","[0.57741445, 0.19582109, 0.35925627, 0.5910297...","[[0.21044426, 0.15949352, 1.9016874, 1.784712,...","[[0.0036610025, 0.29197437, 0.4380889, 0.49628...","[[0.18364595, 0.11826769, 0.32814094, 0.008369...","[[0.075233944, 0.6923916, 0.30742604, 0.203948...","[35280000, 35290000, 35300000, 35310000, 35320..."
3,35238.0,35318,35338.0,35437.0,3.1,Subj 1 then Subj 2,NaN,NaN,NaN,[35237 35255] [35280 35312] [35324 35368] [353...,...,"[0.21442229, nan, nan, nan, 0.24892299, 0.1522...","[0.22374396, nan, nan, nan, 0.28628382, 0.2395...","[0.13560255, nan, nan, nan, 0.25866804, 0.1881...","[0.19912642, nan, nan, nan, 0.23150067, 0.3139...","[0.34216553, nan, nan, nan, 0.2372961, 0.54322...","[[0.7861145, 0.34362876, 0.9110888, 0.39747307...","[[1.8253773, 0.45439005, 0.85494393, 0.3876086...","[[0.8063633, 0.3692385, 0.39918825, 0.4618175,...","[[0.413818, 0.43645024, 0.9202251, 0.4218581, ...","[35280000, 35290000, 35300000, 35310000, 35320..."
4,36337.0,36417,36437.0,36536.0,3.3,Subj 2 Only,NaN,NaN,NaN,[36312 36345] [36439 36503] [36522 36562],...,"[0.17983302, 0.4491682, 0.6623069, 0.38262665,...","[nan, nan, 0.36340868, 0.2605309, 0.48815846, ...","[nan, nan, 0.70878524, 0.36362082, 0.41171372,...","[0.19675834, 0.4006446, 0.3744169, 0.16643988,...","[0.077478655, 0.40043724, 0.51819485, 0.240391...","[[0.63951916, 0.8236137, 0.19508992, 0.2977631...","[[2.5670233, 0.42199114, 0.20445032, 0.0982484...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[0.37562254, 0.17172635, 0.48035538, 0.098667...","[36380000, 36390000, 36400000, 36410000, 36420..."


In [80]:
power_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "power" in col and "timestamps" not in col and "calculation" not in col]

In [81]:
power_columns

['BLA_power_all_frequencies_all_windows',
 'BLA_power_gamma',
 'BLA_power_theta',
 'LH_power_all_frequencies_all_windows',
 'LH_power_gamma',
 'LH_power_theta',
 'MD_power_all_frequencies_all_windows',
 'MD_power_gamma',
 'MD_power_theta',
 'mPFC_power_all_frequencies_all_windows',
 'mPFC_power_gamma',
 'mPFC_power_theta',
 'vHPC_power_all_frequencies_all_windows',
 'vHPC_power_gamma',
 'vHPC_power_theta']

In [82]:
for col in sorted(power_columns):
    updated_item_col = "trial_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_and_post_power_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)


trial_and_post_BLA_power_all_frequencies_all_windows
trial_and_post_BLA_power_gamma
trial_and_post_BLA_power_theta
trial_and_post_LH_power_all_frequencies_all_windows
trial_and_post_LH_power_gamma
trial_and_post_LH_power_theta
trial_and_post_MD_power_all_frequencies_all_windows
trial_and_post_MD_power_gamma
trial_and_post_MD_power_theta
trial_and_post_mPFC_power_all_frequencies_all_windows
trial_and_post_mPFC_power_gamma
trial_and_post_mPFC_power_theta
trial_and_post_vHPC_power_all_frequencies_all_windows
trial_and_post_vHPC_power_gamma
trial_and_post_vHPC_power_theta


In [83]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=power_columns + ["power_timestamps"], errors="ignore")

In [84]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

tone_start_frame
reward_start
reward_dispensed
tone_stop_frame
condition 
competition_closeness
get_reward_frame
out_reward_frame
notes
box_1_port_entry_frames
box_2_port_entry_frames
video_name
tone_start_timestamp
tone_stop_timestamp
box_1_port_entry_timestamps
box_2_port_entry_timestamps
current_subject
agent_tail_base
agent_to_reward_port_angle
color
comp_id
subject_tail_base
subject_to_reward_port_angle
to_reward_port_angle_diff
to_reward_port_angle_sum
tone_start_to_stop_frame
within_trial_index
agent
all_subjects
body_parts
box_bottom_left
box_bottom_right
box_number
box_top_left
box_top_right
experiment
first_timestamp
last_timestamp
reward_port
session_dir
sleap_name
start_frame
stop_frame
tracked_subject
video_id
BLA_gamma_band
BLA_gamma_phase
BLA_lfp_trace
BLA_theta_band
BLA_theta_phase
LH_gamma_band
LH_gamma_phase
LH_lfp_trace
LH_theta_band
LH_theta_phase
MD_gamma_band
MD_gamma_phase
MD_lfp_trace
MD_theta_band
MD_theta_phase
coherence_calculation_frequencies
cohort
granger_

# Filtering out phase

In [85]:
lfp_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "trace" in col and "timestamps" not in col and "calculation" not in col]

In [86]:
band_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "band" in col and "timestamps" not in col and "calculation" not in col]

In [87]:
phase_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "phase" in col and "timestamps" not in col and "calculation" not in col]

In [88]:
phase_columns = phase_columns + band_columns +lfp_columns

In [89]:
phase_columns

['BLA_gamma_phase',
 'BLA_theta_phase',
 'LH_gamma_phase',
 'LH_theta_phase',
 'MD_gamma_phase',
 'MD_theta_phase',
 'mPFC_gamma_phase',
 'mPFC_theta_phase',
 'vHPC_gamma_phase',
 'vHPC_theta_phase',
 'BLA_gamma_band',
 'BLA_theta_band',
 'LH_gamma_band',
 'LH_theta_band',
 'MD_gamma_band',
 'MD_theta_band',
 'mPFC_gamma_band',
 'mPFC_theta_band',
 'vHPC_gamma_band',
 'vHPC_theta_band',
 'BLA_lfp_trace',
 'LH_lfp_trace',
 'MD_lfp_trace',
 'mPFC_lfp_trace',
 'vHPC_lfp_trace']

In [90]:
for col in sorted(phase_columns):
    updated_item_col = "trial_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_and_post_lfp_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[0], axis=1)


trial_and_post_BLA_gamma_band
trial_and_post_BLA_gamma_phase
trial_and_post_BLA_lfp_trace
trial_and_post_BLA_theta_band
trial_and_post_BLA_theta_phase
trial_and_post_LH_gamma_band
trial_and_post_LH_gamma_phase
trial_and_post_LH_lfp_trace
trial_and_post_LH_theta_band
trial_and_post_LH_theta_phase
trial_and_post_MD_gamma_band
trial_and_post_MD_gamma_phase
trial_and_post_MD_lfp_trace
trial_and_post_MD_theta_band
trial_and_post_MD_theta_phase
trial_and_post_mPFC_gamma_band
trial_and_post_mPFC_gamma_phase
trial_and_post_mPFC_lfp_trace
trial_and_post_mPFC_theta_band
trial_and_post_mPFC_theta_phase
trial_and_post_vHPC_gamma_band
trial_and_post_vHPC_gamma_phase
trial_and_post_vHPC_lfp_trace
trial_and_post_vHPC_theta_band
trial_and_post_vHPC_theta_phase


In [91]:
TRIALS_AND_SPECTRAL_DF["trial_and_post_lfp_timestamps"]

0      [34276300, 34276320, 34276340, 34276360, 34276...
1      [34276300, 34276320, 34276340, 34276360, 34276...
2      [35276320, 35276340, 35276360, 35276380, 35276...
3      [35276320, 35276340, 35276360, 35276380, 35276...
4      [36376340, 36376360, 36376380, 36376400, 36376...
                             ...                        
283    [57964900, 57964920, 57964940, 57964960, 57964...
284    [60164920, 60164940, 60164960, 60164980, 60165...
285    [60164920, 60164940, 60164960, 60164980, 60165...
286    [61264940, 61264960, 61264980, 61265000, 61265...
287    [61264940, 61264960, 61264980, 61265000, 61265...
Name: trial_and_post_lfp_timestamps, Length: 288, dtype: object

In [92]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=phase_columns + ["lfp_timestamps"], errors="ignore")

In [93]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [94]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

tone_start_frame
reward_start
reward_dispensed
tone_stop_frame
condition 
competition_closeness
get_reward_frame
out_reward_frame
notes
box_1_port_entry_frames
box_2_port_entry_frames
video_name
tone_start_timestamp
tone_stop_timestamp
box_1_port_entry_timestamps
box_2_port_entry_timestamps
current_subject
agent_tail_base
agent_to_reward_port_angle
color
comp_id
subject_tail_base
subject_to_reward_port_angle
to_reward_port_angle_diff
to_reward_port_angle_sum
tone_start_to_stop_frame
within_trial_index
agent
all_subjects
body_parts
box_bottom_left
box_bottom_right
box_number
box_top_left
box_top_right
experiment
first_timestamp
last_timestamp
reward_port
session_dir
sleap_name
start_frame
stop_frame
tracked_subject
video_id
coherence_calculation_frequencies
cohort
granger_calculation_frequencies
power_calculation_frequencies
recording
session_path
subject
tone_frames
tone_timestamps
video_timestamps
baseline_start_timestamp
post_trial_end_timestamp
video_frame
trial_and_post_agent_locat

In [95]:
TRIALS_AND_SPECTRAL_DF["trial_and_post_kmeans_cluster"].iloc[0].shape

(399,)

In [96]:
raise ValueError()

ValueError: 

# Filtering for rows that are in the video

In [ ]:
TRIALS_AND_SPECTRAL_DF["in_video"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: x["start_frame"] <= x["trial_video_frame"][0] <= x["stop_frame"], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["in_video"]].reset_index()

In [ ]:
TRIALS_AND_SPECTRAL_DF

In [ ]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
raise ValueError()

# Filtering out spikes

In [ ]:
raise ValueError()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["tone_start_timestamp"], stop=x["baseline_stop_timestamp"]).astype(int), axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"]).astype(int), axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_stop_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"].iloc[0]

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1], axis=1)
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1].T, axis=1)
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=["spike_clusters", "spike_times", "neuron_average_fr", "neuron_average_timestamps",], errors="ignore")

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"].iloc[0].shape

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"].iloc[0].shape